In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

% matplotlib inline

# Question 1: Propensity score matching

## 1.1 A naive analysis
We start by importing the data and then split them into 2 subgroups: control and treated. We display some informations on the distributions and some plots.

*NOTE: we play dumb on purpose for this first "naive" analysis. A very succinct analysis of the number and distribution is given, without asking ourselves the typical questions that a data scientist should adress when given any dataset.*

Below we import the datasets and apply the pandas function `describe()` to compute the mean, standard deviation, minimum, quartiles, median and maximum to have a better idea of some features of the dataset. We purposely decided to leave the binary features aside as computing the aformentionned figures about those features is not relevant.

In [ ]:
#Display 2 dataframes from the same cell
from IPython.display import display

df = pd.read_csv('lalonde.csv')

#Create the 2 groups
treated = df.loc[df['treat'] == 1]
control = df.loc[df['treat'] == 0]

# display floats in dataframe with only 2 digits after comma
pd.options.display.float_format = '{:,.2f}'.format

#Informations on both distributions while dropping non-relevant columns to describe
display(treated.drop(['id', 'treat', 'black', 'hispan', 'married', 'nodegree'], axis=1).describe())
display(control.drop(['id', 'treat', 'black', 'hispan', 'married', 'nodegree'], axis=1).describe())

From the figures computed, we see that the two groups do not have the same number of participants. We see that the average age of participants is 25.82 for treated group, and 28.03 for control group, which is slighty different but still rather similar. The mean of years studied is almost the same in the two groups. Regarding the incomes, we can see that on average, before treatment (re74 and re 75 entries) the salary of the treated group was lower than the average incomes of the control group. <br>
Finally, from the the incomes of 78 (re78), we see that after treatment, the difference of average salary is strongly reduced between the two groups. What is interesting to see is that the range of salary for control group is rather small : from 0 (no incomes-unemployed) to 25'564.67\$ whereas for the treated group, the range is way bigger and goes from 0 to 60'307.93\$. 

In [ ]:
# Plot the distribution of the two groups
fig, axes = plt.subplots(2,2, figsize=(16,6))
plt.suptitle('Distribution of incomes')
axes[0,0].set_title('Treated group');
sns.distplot(treated['re78'], kde=False, ax=axes[0,0], bins=30)
sns.kdeplot(treated['re78'], ax=axes[0,1])
axes[1,0].set_title('Control group')
sns.distplot(control['re78'], kde=False, ax=axes[1,0], bins=30)
sns.kdeplot(control['re78'], ax=axes[1,1])
fig.tight_layout(rect=[0, 0.03, 1, 0.95])

From a naive analysis we can see that the controled group seems to have a more spread distribution within its range, but that could be related to the higher number of samples. In comparison, the treated group has many subjects whose salary is in the range 0-15'000\$ and few subject with very high salaries. In both cases, the mean is on the right side of the median and we can therefore conclude that they are both positively skewed. 

## 1.2 A closer look at the data

### Distributions from the treated group

One should notice that we should analyse binary features (black, hispan, married, nodegree) differently than non-binary features (age, educ, re74, re75 and re78). We will begin by looking further into the non binary features.
#### Non-Binary Features
To have a better understanding of the distribution of each non-binary features, we decided to plot violinplot, which countain similar informations as a boxplot (quartiles, median, min/max value) in addition to a kernel density plot. Note that for each plot, the control group appears on the top whereas the the treated group is on the bottom.

In [ ]:
def plot_distributions_nbinary(df, compare_col):
    '''Plot comparative graphs for selected features of both dataframes'''
    plt.figure(figsize=(16,12))
    for i, col in enumerate(compare_col):
        # Violinplot to compare data
        ax = plt.subplot(3, 2, i+1)
        sns.violinplot(data=df, x=compare_col[i], y='treat', ax=ax, inner="quartile", palette='Set2', orient='h');
        ax.set_title('Feature: ' + compare_col[i])
    plt.tight_layout()

In [ ]:
plot_distributions_nbinary(df, ['age', 'educ', 're74', 're75', 're78'])

We can observe some differences between the distributions. The control group seems to be younger, more spread (probably due to the higher number of samples), the revenues in 74, 75 and 78 are a bit higher, at least in mean than the treated group. Nevertheless, the education feature reveals a similar distribution for both groups.
These informations couldn't be seen with the naive analysis above. Now we can focus on the other (binary) features:

#### Binary Features

<font color=red> Nico pas d'accord ici. </font>

In [ ]:
def plot_binary_proportions(df, columns, width = 0.3 ):
    fig, ax = plt.subplots(1, 1, figsize=(10,4))
    r = df.groupby('treat').sum()[columns]
    r = r.div(r.sum(axis=1), axis=0)
    
    ids_ = np.arange(len(columns))
    p1 = plt.bar(ids_, r.iloc[0], width, alpha=0.7, label='control')
    p2 = plt.bar(ids_+width, r.iloc[1], width, alpha=0.7, label='treated')
    
    plt.xlabel('Different classes')
    plt.ylabel('Ratio of people')
    plt.title('Repartition of subject along different categories')
    plt.xticks(ids_ + width / 2, tuple(columns))
    plt.ylim(0,1)
    plt.legend()
    plt.grid()
    plt.tight_layout()
    plt.show()

In [ ]:
df.head()

In [ ]:
plot_binary_proportions(df, ['black', 'hispan', 'married', 'nodegree'])

<font color=red> Observations basées sur l'ancien plot qui n'ont plus de sens ici... </font>

By looking at the distribution of the "binary" features, we see that the racial distribution between the two groups is very different. In the treated group, there is more than 80% of black people and less than 10% of hispanic people, and we can guess that the rest would be white people. In the control group however, there is only 20% of black people and 15% of hispanic, meaning that we have a majority of white people in the control group. As for the other values, the treated group has approximately 20% of marreid people whereas in the control group there is approximately 50% of married subjects. Lastly, the proportion of subjects without a degree is higher in the treated group, approximately 70% against 60% in the control group. <br>
All those differences in proportion from the control group to the treated group raise the problem of population bias (systematic difference of population within the control group vs the treated group) between the two cohorts observed. This bias will skew the analysis as the comparison between the two cohort is not fair. For a fair comparison, it would be necessary to have similar amount of proportion with specific attributes between the two populations observed.

### 1.3 A propensity score model

We use **linear_model** from sklearn in order to define the propensity score for each sample. In order to apply our logistic regression we drop 2 columns for the input: **id** since it doen't contain relevant information and **treat** because it is our output.

In [ ]:
from sklearn import linear_model

logistic = linear_model.LogisticRegression()
X = df.drop(['id','treat'], axis=1)
y = df['treat']
logistic.fit(X, y)
y_pred_proba = logistic.predict_proba(X)

In [ ]:
df['score'] = y_pred_proba[:, 1]
display(df[182:188])

We then have a final dataframe with the propensity score for each row.

### 1.4 Balancing the dataset via matching

We will balance our dataset using the [**linear_sum_assignment**](https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.optimize.linear_sum_assignment.html) from Scipy. This function will allow us to compute the minimum weight matching between the scores from the treated and control group.

In [ ]:
from scipy.optimize import linear_sum_assignment

#Create the 2 groups
treated = df.loc[df['treat'] == 1]
control = df.loc[df['treat'] == 0]

# Create a matrix with distances between every subjects
dist = np.zeros([len(treated['score']), len(control['score'])])
for i, t in enumerate(treated['score']):
    for j, c in enumerate(control['score']):
        dist[i, j] = abs(t-c)

# We use scipy to check that the sum of absolute propensity-score differences 
# between the two matched subjects is minimized.
row_ind, col_ind = linear_sum_assignment(dist)
matched_df = pd.concat([treated, control.iloc[col_ind]])

# The first 185 subjects are from treated groupe and match to the last 
# 185 subjects such as 0 is match with 185, 1 with 186 and so on, n with n+185...
matched_df.iloc[[1, 1+185]]

In [ ]:
# We can check that we have unique matching values (2x185). 
matched_df['id'].nunique()

In [ ]:
# Plots
fig, axes = plt.subplots(2,2, figsize=(16,6))
plt.suptitle('Distribution of incomes after matching')
axes[0,0].set_title('Treated group');
sns.distplot(matched_df.iloc[0:185]['re78'], kde=False, ax=axes[0,0], bins=30)
sns.kdeplot(matched_df.iloc[0:185]['re78'], ax=axes[0,1])
axes[1,0].set_title('Control group')
sns.distplot(matched_df.iloc[185:370]['re78'], kde=False, ax=axes[1,0], bins=30)
sns.kdeplot(matched_df.iloc[185:370]['re78'], ax=axes[1,1])
fig.tight_layout(rect=[0, 0.03, 1, 0.95])

In [ ]:
#Informations on both distributions while dropping irrelevant columns
display(matched_df.iloc[0:185].drop(['id', 'treat', 'black', 'hispan', 'married', 'nodegree'], axis=1).describe())
display(matched_df.iloc[185:370].drop(['id', 'treat', 'black', 'hispan', 'married', 'nodegree'], axis=1).describe())

Now that we have a better dataset, we can see that revenues re78 of the treated group are a bit higher that the control group. Indeed the mean is around 500$ more than the control group and all of the quantiles are also higher.
The treatement may have an effect on the incomes.

In [ ]:
plot_distributions_nbinary(matched_df, ['age', 'educ', 're74', 're75', 're78', 'score'])

In [ ]:
plot_binary_proportions(matched_df, ['black', 'hispan', 'married', 'nodegree'])

We now have a more coherent subset between both groups. Overall profiles are more close to each other. If we look at the shapes of the "violons" we will see similar shapes. Nevertheless it's still not perfect. For example, even if all fields have similar distribution in both sets, some differences stll exist e.i. black people ~32% in control set and ~45% in treated set. Same analysis for age distribution. Treated people are overall older compare to control group.

Even if this propensity score matching is still not the best solution, we can already assume that the programm have a positive effect on the group and its earnings. 


### 1.5 Balancing the groups further

No, we aren't fully satisfied since there are still differences in distributions, mainly between the races.  
One way to solve this issue is to ensure that we are comparing similar subjects. For example it will not make sense to compare an hispanic subject with a black one since there are disparities between races at the time of the survey. Therefore we can check that specific fields are correctly matched, e.i. 'age', 'educ', 'black', 'hispan', 'married', 'nodegree'. We will represent the matching simply as a relative error with 

$$\frac{f_{treated}-f_{control}}{f_{control}}< t \text{, for each feature } f$$

Since age and education are not binary values, we will set a threshold $t$ for the confidence value.



In [ ]:
feat_selected = ['age', 'educ', 'black', 'hispan', 'married', 'nodegree'] # Features used to compare people

df_treated = matched_df.iloc[:185].reset_index()[feat_selected]
df_control = matched_df.iloc[185:].reset_index()[feat_selected]

res = (df_treated-df_control)/(df_control + 1e-5)

In [ ]:
ids_match_new = np.nonzero(np.sum(np.abs(res) > 0.5, axis=1) == 0)[0]
ids_match_new = np.array([ids_match_new, ids_match_new+185]).flatten()

In [ ]:
matched_new_df = matched_df.iloc[ids_match_new]
plot_distributions_nbinary(matched_new_df, ['age', 'educ', 'score'])

In [ ]:
plot_binary_proportions(matched_new_df, ['black', 'hispan', 'married', 'nodegree'])

In [ ]:
# Size of our newly subset: 76 (38 treated and controled)
n_new_match = matched_new_df.shape[0]
n_new_match

As we can see, our newly balanced dataset is way smaller than the previous one. We have 38 matches instead of the previous 185. This is a trade-off between the matching accuracy and the sample size.

### 1.6 A less naive analysis

In [ ]:
plot_distributions_nbinary(matched_new_df, ['re74', 're78'])

In [ ]:
# Plots
fig, axes = plt.subplots(2,2, figsize=(16,6))
plt.suptitle('Distribution of incomes after matching 2.0')
axes[0,0].set_title('Treated group');
sns.distplot(matched_new_df.iloc[0:n_new_match//2]['re78'], kde=False, ax=axes[0,0], bins=30)
sns.kdeplot(matched_new_df.iloc[0:n_new_match//2]['re78'], ax=axes[0,1])
axes[1,0].set_title('Control group')
sns.distplot(matched_new_df.iloc[n_new_match//2:]['re78'], kde=False, ax=axes[1,0], bins=30)
sns.kdeplot(matched_new_df.iloc[n_new_match//2:]['re78'], ax=axes[1,1])
fig.tight_layout(rect=[0, 0.03, 1, 0.95])

In [ ]:
display(matched_new_df[:n_new_match//2].describe())
display(matched_new_df[n_new_match//2:].describe())

As we can see, now that we have a new dataset (smaller but with a better matching), we can see that the programm is definitely worth it. Indeed we can observe that the incomes in re78 are higher for the treated group than for the controled one. In average ~6800\$ compare to ~4900\$ which represent a gap of ~1900\$.

# Question 2: Applied ML

## 2.1 Data loading and features extraction

First we will download the dataset as explained and take a look at the dataset. It is a collection of approximately 20,000 newsgroup documents, partitioned across 20 different newsgroups. For more information go to [this website](http://qwone.com/~jason/20Newsgroups/). The data is composed of multiple fields.

* `data`: is the actual text and description of the news.
* `target`: is the label (as integers) of the news that are part of the 20 categories present in `target_names` (displayed later).
* `filename`: is the location of the file.
* `descritpion`: is the title of the dataset, here : "the 20 newsgroups by date dataset".
* `DESCR` is empty
* `target_names`: is the list of target we are trying to match (news categories linked to `target`).

In [ ]:
newsgroups = fetch_20newsgroups(subset='all')

In [ ]:
list(newsgroups.keys())

Note that some labels are actually sub categories. For example `politics.guns` and `politics.mideast` are both subcategories of `talk`. We can expect it will be more difficult to discriminate and classify news that are part of the same category.

In [ ]:
newsgroups.target_names

We can now create our TF-IDF matrix. The matrix will be sparse and huge since we are not removing any words or limiting the number of features. Here for example the size of vocabulary (features) is 173762.

In [ ]:
tfidf = TfidfVectorizer()
x_tfidf = tfidf.fit_transform(newsgroups.data) 
len(tfidf.vocabulary_)

As asked we will split our data in 3 different sets: `train` (80%), `validation` (10%) and `test` (10%). We will assert our results on the validation set before testing it on the test set. Note that we fixed the seed to 0 for reproducibility purposes.

In [ ]:
ratio_train = 0.8
ratio_validation = 0.1

np.random.seed(0)
id_ = np.random.permutation(np.arange(x_tfidf.shape[0]))
id_train = id_[:np.ceil(x_tfidf.shape[0]*ratio_train).astype(int)]
id_validation = id_[np.floor(x_tfidf.shape[0]*ratio_train).astype(int):
                    np.ceil(x_tfidf.shape[0]*(ratio_train+ratio_validation)).astype(int)]
id_test = id_[np.floor(x_tfidf.shape[0]*(ratio_train+ratio_validation)).astype(int):]

Now that we randomly split our data ids we can create our features sets and labels for classification.

In [ ]:
x_train = x_tfidf[id_train]
x_validation = x_tfidf[id_validation]
x_test = x_tfidf[id_test]

y_train = newsgroups.target[id_train]
y_validation = newsgroups.target[id_validation]
y_test = newsgroups.target[id_test]

## 2.2 Classification using Random Forest

### 2.2.1 Train and Validation set

We decided to go from 0 to around 200 features for both `max_depth` and `n_estimators`. Note that we choosed a logaritmic scale for features numbers since there are no resons to compare values such as for example `max_depth`=63 to `max_depth`=64 since they will give similar results.

In [ ]:
span_depth = np.ceil(np.logspace(0, 2.3, 15)).astype(int)
span_estimators = np.ceil(np.logspace(0, 2.3, 15)).astype(int)

In [ ]:
score = np.zeros((span_depth.shape[0], span_estimators.shape[0]))

for i, n_depth in enumerate(span_depth):
    for j, n_estimator in enumerate(span_estimators):
        clf = RandomForestClassifier(max_depth=n_depth, n_estimators=n_estimator, random_state=0, n_jobs=-1)
        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_validation)
        score[i, j] = accuracy_score(y_pred, y_validation)
    print('Step {}/{} - Best accuracy max_depth={} is {:.4f}'.format(
        i+1, len(span_depth), n_depth, np.max(score[i])))

Since the grid search takes some time to run we saved the results to save time for next runs. Of course you can run the code from scratch at any time to test the results.

In [ ]:
np.save('score_random_forest.npy', 
        {'max_depth': span_depth, 'n_estimators': span_estimators, 'score': score})

In [ ]:
data = np.load('score_random_forest.npy')[()]

We can take a look at the accuracies and we observe that the larger the parameters `n_estimators` and `max_depth` are, the better the accuracy is. However we can also observe that we are reaching some limit. It seems that the algorithm is reaching a plateau around 86%.

In [ ]:
df_score = pd.DataFrame(data['score'])
df_score.columns = [str(num) for num in data['n_estimators']]
df_score.index = [str(num) for num in data['max_depth']]

plt.figure(figsize=(10, 8))
ax = sns.heatmap(df_score, annot=True, fmt=".2f", cmap='RdYlGn')
ax.set_xlabel('n_estimators'); ax.set_ylabel('max_depth');
ax.set_title('Accuracy score over "Max Depth" and "N Estimators"')

Here we can even more clearly see that we reached a plateau. There is only 4% augmentation in accuray for a difference of 169 in # of Estimators. As a consequence, we choosed to not try with higher values since they will not represent a significative improvement in accuracy.

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(df_score.index, df_score.iloc[-1], label='max_d=200')
plt.plot(df_score.index, df_score.iloc[-3], label='max_d=94')
plt.plot(df_score.index, df_score.iloc[-6], label='max_d=31')
plt.xlabel('# Estimators'); plt.ylabel('Accuracy in %'); plt.grid(); 
plt.title('Accuracy on validation set as a funtion of MaxDepth and #Estimators'); plt.legend()

### 2.2.2 Results on Test set

We used the values we best score on validation set e.i. (Max Depth, # Estimators) = (200, 200). We can see that the result on the test set is similar to the one on the validation set with around 86% in accuracy. 

In [ ]:
clf = RandomForestClassifier(max_depth=200, n_estimators=200, random_state=0)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print('Accuracy on train set is {:.4f}'.format(accuracy_score(y_pred, y_test)))

To look more carefuly at the results we can compute the confusion matrix. Note that, by default, the matrix is not normalized and it is difficult to look at coherence of the results. We choosed to normalize it with 

$$\text{precision}_j = \frac{\text{tp}_j}{\text{tp}_j + \sum_{i \neq j} \text{fp}_i} $$

where $\text{tp}_j$ is the amount of true positive and $\text{fp}_i$ are the number of false positive for each class $j$. Note that we did not consider recall in this case.

In [ ]:
cm = confusion_matrix(y_pred, y_test)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

In [ ]:
df_conf = pd.DataFrame(cm)
df_conf.columns = newsgroups.target_names
df_conf.index = newsgroups.target_names

We can see that most classes are performing well with values close to 1. However we can see that some classes such as `soc.religion.christian` only have around 0.7 precision. It is interesting to notice that most of the fp are acually part of `talk.religion.misc` and fewer are `alt.atheism`. Those two topics are indeed related to the first one and we can conclude classification is harder. It is also interesting to notice that `misc.forsale` have fp spread around other classes such as `sci.electronics`, `comp.sys.mac.hardware`, or even `comp.graphics`. Thoses make also sense since this category (forsale) can include a lot of electronic devices and miscellaneous objects.

In [ ]:
plt.figure(figsize=(14, 10))
ax = sns.heatmap(df_conf, annot=True, fmt=".2f", cmap='RdYlGn')

### 2.2.3 Feature Importances
Let's now look at the features more carefuly. If we plot the repartition of the feature importances we can see that there are actually a lot of features that have little importance in classification (close to 0). Moreover there are only around 20 feature that are above 0.002. Note that the y scale is logaritmic. 

In [ ]:
plt.figure(figsize=(10,5))
plt.hist(clf.feature_importances_, bins=50)
plt.yscale('log', nonposy='clip')
plt.xlabel('Feature importance'); plt.ylabel('#Features');
plt.grid(); plt.title('Repartition of feature importance')

As explained before it seems that a small set of top feature have huge impact on clssification. Let's display the 50 with highest importance and therefore look at the relevant words use for classification. It is interessting to notice that some words are actually part of the categories (labels) names. For example "baseball" with `rec.sport.baseball`, "gun" with `talk.politics.gun` or "sale" with `forsale`. However it is wierd to have some unrelevant words such as "are", "you", "for", "the" or even "it" that are often considered as stop words.

In [ ]:
n_features = 50
id_max = np.argsort(clf.feature_importances_)[-n_features:]
np.array(tfidf.get_feature_names())[id_max]